In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#종속변수는 만족도

#가중치 선정

In [2]:
# 필요 라이브러리 불러오기
import pandas as pd
import numpy as np
import os
import seaborn as sns
# 시각화
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc
plt.rcParams['axes.unicode_minus'] = False
# 스케일링
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
# 한글
plt.rc('font', family='NanumBarunGothic')
#OLS
import statsmodels.api as sm

#PCA

In [3]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [4]:
import pandas as pd
data1 = pd.read_csv('/content/drive/MyDrive/데청캠_1분반_4조/최종 csv파일 모음/노인인구/자치구노인인구수_독거포함_최종.csv', encoding='utf8')
data2 = pd.read_csv('/content/drive/MyDrive/데청캠_1분반_4조/최종 csv파일 모음/노인여가만족도/자치구별 만족도.csv', encoding='utf8')

In [5]:
# 변경할 속성이름 정의하기
new_names_columns = {
            '자치구별' : '자치구',
            }

# 속성이름 변경하기
data2.rename(columns=new_names_columns, inplace=True)

In [6]:
data1

,자치구,일반노인수,독거노인수
0,강남구,67665,15432
1,강동구,63647,16059
2,강북구,41861,25195
3,강서구,75796,22863
4,관악구,63112,20308
5,광진구,42695,12159
6,구로구,61276,15983
7,금천구,33041,10731
8,노원구,69509,23770
9,도봉구,53153,14961


In [7]:
#인덱스에 자치구 이름 넣기
data1_1 = data1.set_index('자치구')
data1_1

,일반노인수,독거노인수
자치구,,
강남구,67665,15432
강동구,63647,16059
강북구,41861,25195
강서구,75796,22863
관악구,63112,20308
광진구,42695,12159
구로구,61276,15983
금천구,33041,10731
노원구,69509,23770


In [8]:
#data1 스케일링
# List of column names for which to calculate correlations
column_names = data1_1.columns

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(data1_1), columns=column_names)

In [9]:
# Calculate the correlation matrix
correlation_matrix = scaled_df.corr()
#data1_s=(scaled_df["노인인구수"]).tolist()
#print(data1_s)

In [10]:
#data1 노인인구수 PCA분석 (PCA할때는 독립변수만 들어감!! 종속X)
pca = PCA()
principal_components = pca.fit_transform(scaled_df[data1_1.columns])

# Calculate explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate weights based on explained variance ratio
weights = explained_variance_ratio / np.sum(explained_variance_ratio)
print("PCA-based Weights: ")
for i, weight in enumerate(weights):
    print(f"Variable {i + 1}: {weight:.3f}")

PCA-based Weights: 
Variable 1: 0.855
Variable 2: 0.145


In [11]:
pca_weight = [0.855, 0.145]

In [12]:
scaled_df["일반노인수"].tolist()

[0.7266831827455552,
 0.6681288254153308,
 0.3506412124744972,
 0.8451763334304868,
 0.6603322646458758,
 0.3627951034683765,
 0.6335762168464004,
 0.22210725735937042,
 0.7535558146313027,
 0.5151996502477412,
 0.4417662489070242,
 0.5286359661906149,
 0.37092684348586413,
 0.3696444185368697,
 0.5131157097056251,
 0.27990381812882537,
 0.6179248032643544,
 0.9999999999999999,
 0.5915330807344797,
 0.4809967939376275,
 0.153133197318566,
 0.7571553482949577,
 0.02757213640338091,
 0.0,
 0.5691635091809968]

In [13]:
weighted_df = scaled_df[data1_1.columns].copy()  # Create a copy of the original DataFrame
for i, col in enumerate(data1_1.columns):
    weighted_df[col] = scaled_df[col] * pca_weight[i]

In [14]:
row_sums = weighted_df.sum(axis=1)

# Add the row sums as a new attribute in a new column
weighted_df['자치구별노인인구수점수'] = row_sums

In [15]:
result = weighted_df["자치구별노인인구수점수"]*5

In [20]:
Gu = [
    "강남구", "강동구", "강북구", "강서구", "관악구", "광진구", "구로구", "금천구", "노원구", "도봉구",
    "동대문구", "동작구", "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구", "양천구", "영등포구",
    "용산구", "은평구", "종로구", "중구", "중랑구"
]
print(Gu)

['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


In [21]:
result.tolist()

[3.430359008912682,
 3.205805767761751,
 2.2239911833284753,
 4.2422950197243665,
 3.3470886095722063,
 1.7402331621011449,
 3.0549701374706912,
 1.0801087972893042,
 3.8878905701947772,
 2.506911108822244,
 2.2784200599566806,
 2.553255678716211,
 1.8230776339771173,
 1.8295539454745702,
 2.3071976490153543,
 1.3477366515450546,
 3.08019558984447,
 4.734772134678607,
 2.8402226935216035,
 2.370844391237875,
 0.8032854456313033,
 3.7812260882269007,
 0.1597878993357673,
 0.0,
 2.9142745005584203]

In [22]:
# Create a DataFrame
result_df_pca = pd.DataFrame({'자치구': Gu, '자치구별노인인구수점수': result})

In [23]:
result_df_pca

,자치구,자치구별노인인구수점수
0,강남구,3.430359
1,강동구,3.205806
2,강북구,2.223991
3,강서구,4.242295
4,관악구,3.347089
5,광진구,1.740233
6,구로구,3.054970
7,금천구,1.080109
8,노원구,3.887891
9,도봉구,2.506911


# 회귀분석 - OLS

In [24]:
data2

,자치구,만족도(평균/5점만점)
0,강남구,3.4
1,강동구,3.0
2,강북구,3.0
3,강서구,3.2
4,관악구,3.1
5,광진구,3.3
6,구로구,2.6
7,금천구,3.1
8,노원구,3.3
9,도봉구,2.7


In [25]:
data2 = data2.set_index('자치구')

In [26]:
concatenated_df = pd.concat([data1_1, data2], axis=1)
concatenated_df

,일반노인수,독거노인수,만족도(평균/5점만점)
자치구,,,
강남구,67665,15432,3.4
강동구,63647,16059,3.0
강북구,41861,25195,3.0
강서구,75796,22863,3.2
관악구,63112,20308,3.1
광진구,42695,12159,3.3
구로구,61276,15983,2.6
금천구,33041,10731,3.1
노원구,69509,23770,3.3


In [27]:
#concatenated_df 스케일링
# List of column names for which to calculate correlations
column_names = concatenated_df.columns

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(concatenated_df), columns=column_names)

In [28]:
# Calculate the correlation matrix
correlation_matrix = scaled_df.corr()

In [29]:
# Set up the dependent variable (target) and explanatory variables
y = scaled_df["만족도(평균/5점만점)"]
X = scaled_df[data1_1.columns]

# Add a constant term to the X matrix for the intercept
X = sm.add_constant(X)

# Perform multiple linear regression
model = sm.OLS(y, X).fit()

# Get regression coefficients
coefficients = model.params[1:]  # Exclude the intercept

# Normalize coefficients to get weights
weights = np.abs(coefficients) / np.sum(np.abs(coefficients))

# Print the weights
print("Regression Coefficient-based Weights:")
print(weights)

Regression Coefficient-based Weights:
일반노인수    0.602374
독거노인수    0.397626
dtype: float64


In [30]:
ols_weight = [0.602374, 0.397626]

In [31]:
weighted_df = scaled_df[data1_1.columns].copy()  # Create a copy of the original DataFrame
for i, col in enumerate(data1_1.columns):
    weighted_df[col] = scaled_df[col] * ols_weight[i]

In [32]:
row_sums = weighted_df.sum(axis=1)

# Add the row sums as a new attribute in a new column
weighted_df['자치구별노인인구점수'] = row_sums

In [33]:
result = weighted_df["자치구별노인인구점수"]*5

In [38]:
Gu = [
    "강남구", "강동구", "강북구", "강서구", "관악구", "광진구", "구로구", "금천구", "노원구", "도봉구",
    "동대문구", "동작구", "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구", "양천구", "영등포구",
    "용산구", "은평구", "종로구", "중구", "중랑구"
]
print(Gu)

['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


In [39]:
result.tolist()

[3.0765834665966962,
 2.9708838687252914,
 3.0442157486155637,
 4.270891154943932,
 3.426234220880089,
 1.6117556708249612,
 2.8582512351498446,
 1.0270965561559677,
 4.097154618820905,
 2.386544154062926,
 2.3996610536441763,
 2.3965854795231767,
 1.7680982308654185,
 1.7970293981847951,
 1.857033972323538,
 1.2575187863047632,
 3.063768930070298,
 4.272679343611835,
 2.635608334248812,
 2.311364769544248,
 0.8688278832799473,
 3.7732977204517217,
 0.19799055590400882,
 0.0,
 3.0335435117741367]

In [40]:
# Create a DataFrame
result_df_ols = pd.DataFrame({'자치구': Gu, '자치구별노인인구점수': result})

In [41]:
result_df_ols

,자치구,자치구별노인인구점수
0,강남구,3.076583
1,강동구,2.970884
2,강북구,3.044216
3,강서구,4.270891
4,관악구,3.426234
5,광진구,1.611756
6,구로구,2.858251
7,금천구,1.027097
8,노원구,4.097155
9,도봉구,2.386544


0817에 진행한 회귀

In [ ]:
# 주어진 데이터
data = {
    '자치구별 노인 인구 점수': [5.000000, 4.592647, 4.255588, 4.084991, 3.637919, 3.617682, 3.405235, 3.325105, 3.251930, 3.180320, 3.017805, 2.719840, 2.678992, 2.612708, 2.547426, 2.493547, 2.379147, 1.969226, 1.956508, 1.848248, 1.433754, 1.153957, 0.884936, 0.182438, 0.000000
    ],
    '만족도': [3.4, 3.0, 3.0, 3.2, 3.1, 3.3, 2.6, 3.1, 3.3, 2.7, 2.6, 2.9, 3.0, 3.4, 3.2, 3.4, 3.3, 3.3, 3.0, 3.2, 2.9, 3.0, 2.8, 3.3, 3.1]
}

df = pd.DataFrame(data)

# 독립 변수에 상수 항 추가
X = sm.add_constant(df['자치구별 노인 인구 점수'])

# 회귀 모델 생성
model = sm.OLS(df['만족도'], X)

# 모델 학습
results = model.fit()

# 회귀 분석 결과 출력
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    만족도   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.043
Method:                 Least Squares   F-statistic:                   0.01322
Date:                Thu, 17 Aug 2023   Prob (F-statistic):              0.909
Time:                        13:47:31   Log-Likelihood:                0.80173
No. Observations:                  25   AIC:                             2.397
Df Residuals:                      23   BIC:                             4.834
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             3.0721      0.114     26.896